# Workflow to calculate the quantum efficiency of a polycrystalline material

In [1]:
from ase.calculators.castep import Castep
from ase.atoms import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.build import surface
import ase.calculators.castep
import ase.io.castep

from pymatgen.ext.matproj import MPRester
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.wulff import WulffShape

from functions import calc_surface_energy
from functions import get_qe
from functions import generate_castep_input
from functions import generate_qsub_file

I recommend using pymatgen to generate the bulk/slab and then convert it to an ASE atom object. The advantages of this approach are:
- pymatgen has a better bulk/slab generation algorithm 
- castep can link a calculator to the Atom object


#### Download and save .cif file for necessary materials

In [2]:
with MPRester('API_Key') as m:
    results = m.get_structure_by_material_id("mp-30")
print(type(results))
results.to(filename="Cu_metal_fcc.cif")

<class 'pymatgen.core.structure.Structure'>


#### Generate the bulk structure

In [3]:
structure = ase.io.read(filename="Cu_metal_fcc.cif", format = 'cif')
#view(structure, repeat = (3,3,2))

#### Generate the surface structure

In [4]:
surface_111 = ase.build.surface(lattice = structure, indices = (1,1,1), layers = 5, vacuum=10, tol=1e-10, periodic=True)#
#view(surface, repeat=(5,5,1))
super_surf_111 = surface_111.repeat((3,3,1))
#view(super_surf_111)
surface_100 = ase.build.surface(lattice = structure, indices = (1,0,0), layers = 5, vacuum=10, tol=1e-10, periodic=True)#
#view(surface, repeat=(5,5,1))
super_surf_100 = surface_100.repeat((3,3,1))
#view(super_surf_100)
surface_110 = ase.build.surface(lattice = structure, indices = (1,1,0), layers = 5, vacuum=10, tol=1e-10, periodic=True)#
#view(surface, repeat=(5,5,1))
super_surf_110 = surface_110.repeat((3,3,1))
#view(super_surf_110)

#### Run the calculations

In [5]:
#ASE Atoms object for calculation
struct = surface_111
seed = 'Cu_111' # optional

## Options for CASTEP files
castep_options = {
    'directory': './{}'.format(seed),
    'label': seed,
    # Param File Instructions
    'task': 'SinglePoint',
    'xc_functional': 'PBE',
    'energy_cutoff': 400,
    'opt_strategy': 'Speed',
    'fix_occup' : 'FALSE',
    'spin_polarized': 'FALSE',
    #Cell File Instructions
    'kpoints': (2,2,2),
    'symmetry': 'FALSE',
    'fix_all_cell': 'FALSE'
}

PBS_options = {
    'seed_name': seed,
    'queue': 'short',
    'program': 'castep_19',
    'bandstructure': False
}
print(PBS_options)

generate_castep_input(calc_struct=struct, **castep_options)
generate_qsub_file(**PBS_options)

{'seed_name': 'Cu_111', 'queue': 'short', 'program': 'castep_19', 'bandstructure': True}
['#!/bin/bash  --login\n']


#### Extract the final energies

#### Calculate the surface energy

#### Get the % of each surface from a  Wulff construction

#### Launch OptaDOS

#### Get QE from OptaDOS output

#### Calculate the weighted average QE

#### Calculate and display plenty more properties (DOS, bands, ...)